<a href="https://colab.research.google.com/github/tam8738/KHDL/blob/main/cleanData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/KHDL/data/enron_spam_data.csv')  # Thay đường dẫn tương ứng
print(data.head())  # Hiển thị 5 dòng đầu tiên của dữ liệu

   Message ID                       Subject  \
0           0  christmas tree farm pictures   
1           1      vastar resources , inc .   
2           2  calpine daily gas nomination   
3           3                    re : issue   
4           4     meter 7268 nov allocation   

                                             Message Spam/Ham        Date  
0                                                NaN      ham  1999-12-10  
1  gary , production from the high island larger ...      ham  1999-12-13  
2             - calpine daily gas nomination 1 . doc      ham  1999-12-14  
3  fyi - see note below - already done .\nstella\...      ham  1999-12-14  
4  fyi .\n- - - - - - - - - - - - - - - - - - - -...      ham  1999-12-14  


In [34]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import re

In [35]:
# Kiểm tra giá trị bị thiếu
print(data.isnull().sum())

Message ID      0
Subject       289
Message       371
Spam/Ham        0
Date            0
dtype: int64


In [36]:
#thay thế các dữ liệu bị thiếu trong cột message và subject
data['Message'].fillna('No Message', inplace=True)
data['Subject'].fillna('No Subject', inplace=True)

# Xóa các cột không cần thiết
data = data.drop(columns=['Message ID', 'Date'])

# Kiểm tra lại dữ liệu sau khi xử lý
print(data.isnull().sum())
print(data.head())

Subject     0
Message     0
Spam/Ham    0
dtype: int64
                        Subject  \
0  christmas tree farm pictures   
1      vastar resources , inc .   
2  calpine daily gas nomination   
3                    re : issue   
4     meter 7268 nov allocation   

                                             Message Spam/Ham  
0                                         No Message      ham  
1  gary , production from the high island larger ...      ham  
2             - calpine daily gas nomination 1 . doc      ham  
3  fyi - see note below - already done .\nstella\...      ham  
4  fyi .\n- - - - - - - - - - - - - - - - - - - -...      ham  


<ipython-input-36-74bc9127cf8d>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Message'].fillna('No Message', inplace=True)
<ipython-input-36-74bc9127cf8d>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [37]:
# Kiểm tra số lượng dòng trùng lặp
print(data.duplicated().sum())

3222


In [38]:
# Loại bỏ các dòng trùng lặp
data = data.drop_duplicates()

In [39]:
print(data.isnull().sum())

Subject     0
Message     0
Spam/Ham    0
dtype: int64


In [40]:
#Chuyển nhãn Spam/Ham thành giá trị số
data['Spam/Ham'] = data['Spam/Ham'].map({'ham': 0, 'spam': 1})

In [41]:
# 2. Kết hợp cột 'Subject' và 'Message' thành một cột văn bản duy nhất
data['Text'] = data['Subject'] + " " + data['Message']

In [42]:

# Chuyển thành chữ thường
data['Text'] = data['Text'].str.lower()

# Loại bỏ ký tự đặc biệt
data['Text'] = data['Text'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Loại bỏ khoảng trắng thừa
data['Text'] = data['Text'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Kiểm tra dữ liệu sau khi làm sạch
print(data['Text'].head())

0              christmas tree farm pictures no message
1    vastar resources inc gary production from the ...
2    calpine daily gas nomination calpine daily gas...
3    re issue fyi see note below already done stell...
4    meter nov allocation fyi forwarded by lauri a ...
Name: Text, dtype: object


In [43]:
print(data.head())

                        Subject  \
0  christmas tree farm pictures   
1      vastar resources , inc .   
2  calpine daily gas nomination   
3                    re : issue   
4     meter 7268 nov allocation   

                                             Message  Spam/Ham  \
0                                         No Message         0   
1  gary , production from the high island larger ...         0   
2             - calpine daily gas nomination 1 . doc         0   
3  fyi - see note below - already done .\nstella\...         0   
4  fyi .\n- - - - - - - - - - - - - - - - - - - -...         0   

                                                Text  
0            christmas tree farm pictures no message  
1  vastar resources inc gary production from the ...  
2  calpine daily gas nomination calpine daily gas...  
3  re issue fyi see note below already done stell...  
4  meter nov allocation fyi forwarded by lauri a ...  


In [44]:
print(data.isna().sum())

Subject     0
Message     0
Spam/Ham    0
Text        0
dtype: int64


In [45]:
print(data[data['Text'].str.strip() == ''])

                                                 Subject  \
3789   ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ...   
5078                ? ? ? ? 13 ? ? ? ? ? ? ? * ? * ? * ?   
33649                                                  1   

                                              Message  Spam/Ham Text  
3789              ???? ??? ????? ?????? ?????? ????\n         1       
5078    234567890\n 234567890\n 234567890\n 234567890         1       
33649                                             1\n         1       


In [46]:
data.loc[3789, 'Text'] = "neutral"
data.loc[5078, 'Text'] = "neutral"
data.loc[33649, 'Text'] = "neutral"
print(data[data['Text'].str.strip() == ''])

Empty DataFrame
Columns: [Subject, Message, Spam/Ham, Text]
Index: []


In [47]:
print(data['Spam/Ham'].value_counts())

Spam/Ham
0    15910
1    14584
Name: count, dtype: int64


In [48]:
# Giảm số lượng mẫu của ham để cân bằng với spam
ham_downsampled = resample(ham,
                           replace=False,  # Không lấy lại mẫu
                           n_samples=len(spam),  # Số lượng mẫu bằng với spam
                           random_state=42)

# Kết hợp lại thành DataFrame cân bằng
balanced_data = pd.concat([ham_downsampled, spam])

# Kiểm tra phân phối nhãn sau khi cân bằng
print("Phân phối nhãn sau khi undersampling:")
print(balanced_data['Spam/Ham'].value_counts())

Phân phối nhãn sau khi undersampling:
Spam/Ham
0    14584
1    14584
Name: count, dtype: int64


In [ ]:
data.to_csv("/content/drive/MyDrive/KHDL/data/dataFinal3.csv", index=False)

print("Dữ liệu đã xử lý được lưu vào file 'dataFinal3.csv'.")

Dữ liệu đã xử lý được lưu vào file 'dataFinal3.csv'.
